In [3]:
import numpy as np

# Datos dados
R = 30               # [m]
B = 3                  # [-]
rho = 1.225            # [kg/m^3]
V_0 = 8              # [m/s]
omega = 65           # [rad/s] velocidad de rotación
pitch_angle = -3 * np.pi / 180  # [rad]
beta = 2 * np.pi / 180          # [rad]  (no se usa más adelante, igual que en MATLAB)
chord = 0.5            # [m]
C_lift = 0.5           # [-]
C_drag = 0.01          # [-]
r = 24.5               # [m]
relax_factor = 0.1     # [-]

# Inicializar a, a' y epsilons (típicamente a = a' = 0)
a = 0.3
a_prime = 0.0
epsilon_a = 1.0
epsilon_aprime = 1.0

# Calcular solidez
sigma = chord * B / (2 * np.pi * r)
z = 0  # contador de iteraciones

# Bucle iterativo
while epsilon_a > 1e-10 and epsilon_aprime > 1e-10:

    # Ángulo de flujo
    flow_angle = np.arctan((1 - a) * V_0 / ((1 + a_prime) * omega * r))  # [rad]

    # Ángulo de ataque local
    alpha = flow_angle - pitch_angle  # [rad] (no se usa de momento, igual que en MATLAB)

    # Coeficientes normal y tangencial
    C_n = C_lift * np.cos(flow_angle) + C_drag * np.sin(flow_angle)
    C_t = C_lift * np.sin(flow_angle) - C_drag * np.cos(flow_angle)

    # Coeficiente de empuje y factor de Prandtl
    C_Thrust = ((1 - a) ** 2 * C_n * sigma) / (np.sin(flow_angle) ** 2)
    F = (2 / np.pi) * np.arccos(
        np.exp((-B / 2) * (R - r) / (r * np.sin(abs(flow_angle))))
    )

    # Actualizar a
    if a < 1/3:
        a_aprox = sigma * C_n * (1 - a) / (4 * F * (np.sin(flow_angle) ** 2))
    else:
        a_aprox = C_Thrust / (4 * F * (1 - 0.25 * (5 - 3 * a) * a))

    a_new = relax_factor * a_aprox + (1 - relax_factor) * a

    # Actualizar a'
    a_prime_aprox = (
        sigma * C_t * (1 + a_prime) /
        (4 * F * np.sin(flow_angle) * np.cos(flow_angle))
    )
    a_prime_new = relax_factor * a_prime_aprox + (1 - relax_factor) * a_prime

    # Errores para criterio de convergencia
    epsilon_a = abs(a_new - a)
    epsilon_aprime = abs(a_prime_new - a_prime)

    # Actualizar variables
    a = a_new
    a_prime = a_prime_new

    z += 1

# Cálculo de p_t y p_n
V_rel = (1 - a) * V_0 / np.sin(flow_angle)
p_n = 0.5 * rho * V_rel**2 * chord * C_n  # [N/m]
p_t = 0.5 * rho * V_rel**2 * chord * C_t  # [N/m]

print(f"a      = {a}")
print(f"a'     = {a_prime}")
print(f"iter   = {z}")
print(f"p_n    = {p_n} N/m")
print(f"p_t    = {p_t} N/m")
print(f"V_rel  = {V_rel} m/s")
print(f"Flow angle  = {flow_angle} rad")


a      = 0.08791283494136758
a'     = 0.001031424581898202
iter   = 123
p_n    = 632.9174588509322 N/m
p_t    = 59.35347800324454 N/m
V_rel  = 64.42548764205337 m/s
Flow angle  = 0.11350147101669746 rad
